# **World Bank Social Development Dataset:** *ETL*
### Source: [World Bank Social Development](https://data.worldbank.org/topic/social-development?view=chart)

#### Our questions:
<ol>
<li>How do the indicators differ in countries, especially in those that are labeled as low income?</li>
<li>What is the labor force participation rate per region?</li>
<li>What is the labor force participation rate per income group?</li>
<li>In regions that show children under the age of 14 working and attending school, what is their literacy rate upon entering adolescence?</li>
</ol>


In [3]:
# Imports
import pandas as pd

### **(E)xtract the Data:**
**REQUIRED:** *Download the data from [here](https://data.worldbank.org/topic/social-development?view=chart) and save as a CSV in `data` folder.

In [4]:
# Load in csv
data = pd.read_csv('data/API_15_DS2_en_csv_v2_4254232.csv', skiprows=4)
region_codes = pd.read_csv('data/Metadata_Country_API_15_DS2_en_csv_v2_4254232.csv')

# Merge dataset and region data files together
data = data.merge(region_codes, how='inner', on='Country Code')

# Get a couple of lines of data to make sure loaded correctly
print(data.head())

  Country Name Country Code  \
0        Aruba          ABW   
1        Aruba          ABW   
2        Aruba          ABW   
3        Aruba          ABW   
4        Aruba          ABW   

                                      Indicator Name     Indicator Code  \
0             Life expectancy at birth, male (years)  SP.DYN.LE00.MA.IN   
1           Life expectancy at birth, female (years)  SP.DYN.LE00.FE.IN   
2  Adolescent fertility rate (births per 1,000 wo...        SP.ADO.TFRT   
3  Refugee population by country or territory of ...     SM.POP.REFG.OR   
4  Refugee population by country or territory of ...        SM.POP.REFG   

       1960      1961    1962     1963     1964     1965  ...     2018  \
0   64.0840   64.4290  64.747  65.0530  65.3560  65.6600  ...  73.6280   
1   67.1290   67.6250  68.065  68.4580  68.8170  69.1650  ...  78.5070   
2  106.2062  102.8116  99.417  94.4542  89.4914  84.5286  ...  21.1736   
3       NaN       NaN     NaN      NaN      NaN      NaN  ...     

### **(T)ransform the Data:**
- Remove **nonessential** columns: `Indicator Code`, `Country Code`, `SpecialNotes`, `TableName`
- Convert individaul year columns to single **`Year`** column
- Handle **`na`** values
- Cast **typing** to values
- **Rename** `Country Name` and `Indicator Name` to `Country` and `Indicator` to make consistent

In [5]:
# Remove nonessential columns
data = data.drop(columns=['Indicator Code', 'Country Code', 'SpecialNotes', 'TableName'])

# Get a couple of lines of data to make sure updated correctly
print(data.head())

  Country Name                                     Indicator Name      1960  \
0        Aruba             Life expectancy at birth, male (years)   64.0840   
1        Aruba           Life expectancy at birth, female (years)   67.1290   
2        Aruba  Adolescent fertility rate (births per 1,000 wo...  106.2062   
3        Aruba  Refugee population by country or territory of ...       NaN   
4        Aruba  Refugee population by country or territory of ...       NaN   

       1961    1962     1963     1964     1965     1966    1967  ...     2016  \
0   64.4290  64.747  65.0530  65.3560  65.6600  65.9590  66.242  ...  73.3480   
1   67.6250  68.065  68.4580  68.8170  69.1650  69.5260  69.917  ...  78.2370   
2  102.8116  99.417  94.4542  89.4914  84.5286  79.5658  74.603  ...  23.8416   
3       NaN     NaN      NaN      NaN      NaN      NaN     NaN  ...      NaN   
4       NaN     NaN      NaN      NaN      NaN      NaN     NaN  ...      NaN   

     2017     2018     2019     2020  

In [6]:
# Convert individual year columns into one column
data_new = data[['Country Name', 'Indicator Name', 'Region', 'IncomeGroup']]
years = []
for year in range(1960, 2022):
    this_year = data_new.copy()
    this_year['Year'] = year
    this_year['Value'] = data[str(year)]
    years.append(this_year)
data = pd.concat(years)

# Get a couple of lines of data to make sure updated correctly
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558620 entries, 0 to 9009
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Country Name    558620 non-null  object 
 1   Indicator Name  558620 non-null  object 
 2   Region          457436 non-null  object 
 3   IncomeGroup     455328 non-null  object 
 4   Year            558620 non-null  int64  
 5   Value           200412 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 29.8+ MB
None


In [7]:
# Handle na values

# Region
data['Region'] = data['Region'].fillna('N/A')

# Region
data['IncomeGroup'] = data['IncomeGroup'].fillna('N/A')

# Value: leave na values as na

# Check to see if handled
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558620 entries, 0 to 9009
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Country Name    558620 non-null  object 
 1   Indicator Name  558620 non-null  object 
 2   Region          558620 non-null  object 
 3   IncomeGroup     558620 non-null  object 
 4   Year            558620 non-null  int64  
 5   Value           200412 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 29.8+ MB
None


In [8]:
# Ensure values are cast correctly
data['Country Name'] = data['Country Name'].astype(str)
data['Indicator Name'] = data['Indicator Name'].astype(str)
data['Region'] = data['Region'].astype(str)
data['IncomeGroup'] = data['IncomeGroup'].astype(str)

In [9]:
# Rename to get consistent scheming
data = data.rename(columns={'Country Name': 'Country', 'Indicator Name': 'Indicator'})

# Make sure changed
data.head()

,Country,Indicator,Region,IncomeGroup,Year,Value
0,Aruba,"Life expectancy at birth, male (years)",Latin America & Caribbean,High income,1960,64.0840
1,Aruba,"Life expectancy at birth, female (years)",Latin America & Caribbean,High income,1960,67.1290
2,Aruba,"Adolescent fertility rate (births per 1,000 wo...",Latin America & Caribbean,High income,1960,106.2062
3,Aruba,Refugee population by country or territory of ...,Latin America & Caribbean,High income,1960,NaN
4,Aruba,Refugee population by country or territory of ...,Latin America & Caribbean,High income,1960,NaN


### **(L)oad the Data:**
The file is saved to: `data/data-cleaned.csv`

In [10]:
# Save into new cleaned CSV file
data.to_csv('data/data-cleaned.csv')